In [1]:
import trafilatura
import requests
import lxml
from elasticsearch import Elasticsearch


In [2]:
vnexpress = {
    "Đời sống":
    ["https://vnexpress.net/doi-song/cooking",
     "https://vnexpress.net/doi-song/to-am",
     "https://vnexpress.net/doi-song/bai-hoc-song",
     "https://vnexpress.net/doi-song/nha",
     "https://vnexpress.net/doi-song/tieu-dung"],
    "Thời sự":
    ["https://vnexpress.net/thoi-su/chinh-tri",
     "https://vnexpress.net/thoi-su/dan-sinh",
     "https://vnexpress.net/thoi-su/giao-thong",
     "https://vnexpress.net/thoi-su/mekong",
     "https://vnexpress.net/thoi-su/quy-hy-vong"],
    "Thế giới":
    ["https://vnexpress.net/the-gioi/quan-su",
     "https://vnexpress.net/the-gioi/tu-lieu",
     "https://vnexpress.net/the-gioi/phan-tich",
     "https://vnexpress.net/the-gioi/nguoi-viet-5-chau",
     "https://vnexpress.net/the-gioi/cuoc-song-do-day"],
    "Podcast":
    ["https://vnexpress.net/podcast",
     "https://vnexpress.net/podcast/vnexpress-hom-nay",
     "https://vnexpress.net/podcast/ly-hon",
     "https://vnexpress.net/podcast/tham-thi",
     "https://vnexpress.net/podcast/ban-on-khong",
     "https://vnexpress.net/podcast/toi-trong-guong",
     "https://vnexpress.net/podcast/chuyen-di-lam",
     "https://vnexpress.net/podcast/up-mo-18"],
    "Kinh doanh":
    ["https://vnexpress.net/kinh-doanh",
     "https://vnexpress.net/kinh-doanh/quoc-te",
     "https://vnexpress.net/kinh-doanh/doanh-nghiep",
     "https://vnexpress.net/kinh-doanh/chung-khoan",
     "https://vnexpress.net/kinh-doanh/bat-dong-san",
     "https://vnexpress.net/kinh-doanh/ebank",
     "https://vnexpress.net/kinh-doanh/vi-mo",
     "https://vnexpress.net/kinh-doanh/tien-cua-toi",
     "https://vnexpress.net/kinh-doanh/bao-hiem",
     "https://vnexpress.net/kinh-doanh/hang-hoa",
     "https://vnexpress.net/kinh-doanh/e-commerce-40", ],
    "Khoa học":
    ["https://vnexpress.net/khoa-hoc",
     "https://vnexpress.net/khoa-hoc/khoa-hoc-trong-nuoc",
     "https://vnexpress.net/khoa-hoc/tin-tuc",
     "https://vnexpress.net/khoa-hoc/phat-minh",
     "https://vnexpress.net/khoa-hoc/ung-dung",
     "https://vnexpress.net/khoa-hoc/the-gioi-tu-nhien",
     "https://vnexpress.net/khoa-hoc/thuong-thuc",
     "https://vnexpress.net/khoa-hoc/cuoc-thi-sang-kien-khoa-hoc", ],
    "Giải trí":
    ["https://vnexpress.net/giai-tri",
     "https://vnexpress.net/giai-tri/gioi-sao",
     "https://vnexpress.net/giai-tri/phim",
     "https://vnexpress.net/giai-tri/nhac",
     "https://vnexpress.net/giai-tri/thoi-trang",
     "https://vnexpress.net/giai-tri/lam-dep",
     "https://vnexpress.net/giai-tri/sach",
     "https://vnexpress.net/giai-tri/san-khau-my-thuat", ],
    "Thể thao":
    ["https://vnexpress.net/the-thao",
     "https://vnexpress.net/bong-da",
     "https://vnexpress.net/the-thao/du-lieu-bong-da",
     "https://vnexpress.net/the-thao/tennis",
     "https://vnexpress.net/the-thao/bundesliga",
     "https://vnexpress.net/the-thao/cac-mon-khac",
     "https://vnexpress.net/the-thao/world-cup-2022",
     "https://vnexpress.net/the-thao/v-league",
     "https://vnexpress.net/the-thao/hau-truong",
     "https://vnexpress.net/the-thao/photo",
     "https://vnexpress.net/the-thao/tuong-thuat", ],
    "Pháp luật":
    ["https://vnexpress.net/phap-luat",
     "https://vnexpress.net/phap-luat/ho-so-pha-an",
     "https://vnexpress.net/phap-luat/tu-van ", ],
    "Giáo dục":
    ["https://vnexpress.net/giao-duc",
     "https://vnexpress.net/giao-duc/tin-tuc",
     "https://vnexpress.net/giao-duc/tuyen-sinh",
     "https://vnexpress.net/giao-duc/chan-dung",
     "https://vnexpress.net/giao-duc/du-hoc",
     "https://vnexpress.net/giao-duc/hoc-tieng-anh",
     "https://vnexpress.net/giao-duc/trac-nghiem",
     "https://vnexpress.net/giao-duc/giao-duc-40", ],
    "Sức khỏe":
    ["https://vnexpress.net/suc-khoe",
     "https://vnexpress.net/suc-khoe/tin-tuc",
     "https://vnexpress.net/suc-khoe/tu-van",
     "https://vnexpress.net/suc-khoe/dinh-duong",
     "https://vnexpress.net/suc-khoe/khoe-dep",
     "https://vnexpress.net/suc-khoe/dan-ong",
     "https://vnexpress.net/suc-khoe/cac-benh",
     "https://vnexpress.net/suc-khoe/vaccine",
     "https://vnexpress.net/suc-khoe/di-chung-covid", ],
    "Du lịch":
    ["https://vnexpress.net/du-lich",
     "https://vnexpress.net/du-lich/diem-den",
     "https://vnexpress.net/du-lich/am-thuc",
     "https://vnexpress.net/du-lich/dau-chan",
     "https://vnexpress.net/du-lich/tu-van",
     "https://vnexpress.net/du-lich/anh-video",
     "https://vnexpress.net/du-lich/an-va-choi", ],
    "Số hóa":
    ["https://vnexpress.net/so-hoa",
     "https://vnexpress.net/so-hoa/cong-nghe",
     "https://vnexpress.net/so-hoa/san-pham",
     "https://vnexpress.net/so-hoa/blockchain", ],
    "Ô tô":
    ["https://vnexpress.net/oto-xe-may",
     "https://vnexpress.net/oto-xe-may/thi-truong",
     "https://vnexpress.net/oto-xe-may/cam-lai",
     "https://vnexpress.net/oto-xe-may/dien-dan",
     "https://vnexpress.net/oto-xe-may/car-awards-2021", ]
}


In [3]:
for key, value in vnexpress.items():
    url_array = []
    for url in value:
        for i in range(2, 10):
            url_array.append(url + "-p" + str(i))
    vnexpress[key]+=url_array


### Lấy link bài viết từ các link chuyên mục trên

In [4]:
vnexpressLinks = {}
for key in vnexpress:
    vnexpressLinks[key] = []
for key, value in vnexpress.items():
    for url in value:
        try:
            response = requests.get(url)
            tree = lxml.html.fromstring(response.content)
            title_elem = tree.cssselect('.title-news a')
            for elem in title_elem:
                vnexpressLinks[key].append(elem.get('href'))
        except:
            pass
for key in vnexpressLinks:
    print(key, len(vnexpressLinks[key]))


Đời sống 1368
Thời sự 1336
Thế giới 1350
Podcast 923
Kinh doanh 2729
Khoa học 1885
Giải trí 2181
Thể thao 3235
Pháp luật 554
Giáo dục 2146
Sức khỏe 2888
Du lịch 1620
Số hóa 1144
Ô tô 1098


### Lấy phần nội dung trong các bài viết

In [5]:
vnexpressContent = {}
for key in vnexpress:
    vnexpressContent[key] = []
for key, value in vnexpressLinks.items():
    for url in value:
        if(url.startswith('https://vnexpress.net/')):
            try:
                id = url.split('-')[-1].replace('.html', '')
                downloaded = trafilatura.fetch_url(url)
                tree = lxml.html.fromstring(str(downloaded))
                date_elem = len(tree.cssselect(".date")) and tree.cssselect(".date")[0].text or ""
                content = trafilatura.extract(downloaded)
                content = content.replace('\n', ' ')
                item = {"domain": "https://vnexpress.net", "URL": url,"content": content, "date": date_elem,"id": id}
                vnexpressContent[key].append(item)
            except:
                pass
for key in vnexpressContent:
    print(key, len(vnexpressContent[key]))

ERROR:root:pycurl: https://vnexpress.net/mac-benh-la-nguoi-phu-nu-viet-khuyen-chong-lay-vo-khac-4397184.html (28, 'Failed to connect to vnexpress.net port 443 after 21029 ms: Timed out')
ERROR:root:pycurl: https://vnexpress.net/trung-doc-dac-110-trieu-usd-sau-32-nam-chi-mua-mot-bo-so-4457486.html (28, 'Failed to connect to vnexpress.net port 443 after 21034 ms: Timed out')
ERROR:trafilatura.core:not enough text None
ERROR:trafilatura.core:not enough text None
ERROR:trafilatura.core:not enough text None
ERROR:trafilatura.core:not enough text None
ERROR:trafilatura.core:not enough text None
ERROR:trafilatura.core:not enough text None
ERROR:trafilatura.core:not enough text None
ERROR:root:pycurl: https://vnexpress.net/xay-nha-phong-theo-den-taj-mahal-tang-vo-4393377.html (28, 'Failed to connect to vnexpress.net port 443 after 21055 ms: Timed out')
ERROR:trafilatura.core:not enough text None
ERROR:trafilatura.core:not enough text None
ERROR:trafilatura.core:not enough text None
ERROR:trafi

In [23]:
client = Elasticsearch("http://localhost:9200")

client.info()


{'name': 'HoangPhiLong-20182651',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'jF2zfdDdQqueQik2u6h9Ug',
 'version': {'number': '7.17.2',
  'build_flavor': 'default',
  'build_type': 'deb',
  'build_hash': 'de7261de50d90919ae53b0eff9413fd7e5307301',
  'build_date': '2022-03-28T15:12:21.446567561Z',
  'build_snapshot': False,
  'lucene_version': '8.11.1',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [27]:
def gendata():
    for key,value in vnexpressContent.items():
         for item in value:
            temp_item = {k: item[k] for k in set(list(item.keys())) - set(['id'])}
            temp_item['category'] = key
            client.index(index="journal", id=item['id'], document=temp_item)

gendata()

/home/hplong/.local/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
